In [ ]:
from urllib.parse import urljoin, urldefrag, unquote, urlparse, urlunparse
 
from bs4 import BeautifulSoup
import requests

In [ ]:
FORBIDDEN_LIST = ["index.php", "Файл:", "Шаблон:", "Категория:", "Служебная:", "Обсуждение:", "Википедия:", "Портал:", "Участник:", "Обсуждение_участника:", "Проект:", "Заглавная_страница"]
link_first = "https://ru.wikipedia.org/wiki/Уравнение_теплопроводности"
link_second = "https://ru.wikipedia.org/wiki/Тензор"
link_second = "https://ru.wikipedia.org/wiki/Хинкали"

# link_first = "https://ru.wikipedia.org/wiki/Полный_пока"
# link_second = "https://ru.wikipedia.org/wiki/Сон_в_красном_тереме"

# link_first = "https://ru.wikipedia.org/wiki/Теория_категорий"
link_second = "https://ru.wikipedia.org/wiki/Идиот_(роман)"

link_first = "https://ru.wikipedia.org/wiki/Чемпионат_Норвегии_по_кёрлингу_среди_смешанных_команд_2019"
link_second = "https://ru.wikipedia.org/wiki/Александр_Иванович"

In [ ]:
cache = {}

In [ ]:
%%time

def get_neighbours(vertex):
    if vertex in cache:
        return cache[vertex]
    neighbours = []
    html = requests.get(vertex).content.decode("utf-8")
    parser = BeautifulSoup(html, "html.parser")
    for a_tag in filter(lambda a_tag: "href" in a_tag.attrs, parser("a")):
        url = unquote(urldefrag(urljoin(vertex, a_tag["href"]))[0])
        q = urlparse(url)
        q = q._replace(query="")
        url = urlunparse(q)
        if not "ru.wikipedia" in url:
            continue
        if any(word in url for word in FORBIDDEN_LIST):
            continue
        neighbours.append(url)
    neighbours = list(set(neighbours))
    cache[vertex] = neighbours
    return neighbours

lists = [
    {
        "dir": "out",
        "visited": {link_first},
        "layer": 0,
        0: {
            "vertexes": [link_first],
            "from": [None]
        }
    },
    {
        "dir": "in",
        "visited": {link_second},
        "layer": 0,
        0: {
            "vertexes": [link_second],
            "from": [None]
        }
    }
]

def process(vertex, side):
    neighbours = []
    if lists[side]["dir"] == "out":
        neighbours = get_neighbours(vertex)
    else:
        title = urlparse(vertex).path[6:]
        here_url = f"https://ru.wikipedia.org/w/index.php?title=Служебная:Ссылки_сюда/{title}&limit=5000"
        neighbours = get_neighbours(here_url)
    vertexes = []
    froms = []
    for url in neighbours:
        if url not in lists[side]["visited"]:
            lists[side]["visited"] |= {url}
            vertexes.append(url)
            froms.append(vertex)
    return vertexes, froms

def found_path():
    reachable_from_first = set(lists[0][lists[0]["layer"]]["vertexes"])
    reachable_from_second = set(lists[1][lists[1]["layer"]]["vertexes"])
    return len(reachable_from_first & reachable_from_second) > 0

while not found_path():
    side = 0 if len(lists[0][lists[0]["layer"]]["vertexes"]) < len(lists[1][lists[1]["layer"]]["vertexes"]) else 1
    vertexes = lists[side][lists[side]["layer"]]["vertexes"]
    neighbours = []
    froms = []
    for vertex in vertexes:
        neighbours_d, froms_d = process(vertex, side)
        neighbours += neighbours_d
        froms += froms_d
    lists[side]["layer"] += 1
    lists[side][lists[side]["layer"]] = {
        "vertexes": neighbours,
        "from": froms
    }
    vertexes_visited = sum(
        sum(
            len(lists[side][layer]["vertexes"])
            for layer in range(lists[side]["layer"] + 1)
        )
        for side in range(2)
    )
    print(vertexes_visited)

vertexes_visited = sum(
    sum(
        len(lists[side][layer]["vertexes"])
        for layer in range(lists[side]["layer"] + 1)
    )
    for side in range(2)
)
print(f"Посещено вершин: {vertexes_visited}")
if found_path():
    reachable_from_first = set(lists[0][lists[0]["layer"]]["vertexes"])
    reachable_from_second = set(lists[1][lists[1]["layer"]]["vertexes"])
    intersection = reachable_from_first & reachable_from_second
    print(f"Найдено путей: {len(intersection)}")
    print("Через:")
    for middlepoint in intersection:
        print(f"    {middlepoint}")
    print()
    for middlepoint in intersection:
        path = []
        v = middlepoint
        layer = lists[0]["layer"]
        while v != link_first:
            i = lists[0][layer]["vertexes"].index(v)
            v = lists[0][layer]["from"][i]
            path = [v] + path
            layer -= 1

        v = middlepoint
        layer = lists[1]["layer"]
        while v != link_second:
            i = lists[1][layer]["vertexes"].index(v)
            path.append(v)
            v = lists[1][layer]["from"][i]
            layer -= 1
        path.append(v)
        print("\n".join(path))
        print()
else:
    print("Путь не найден")

26
47
2900
4841
Посещено вершин: 4841
Найдено путей: 17
Через:
    https://ru.wikipedia.org/wiki/Сергий_Радонежский
    https://ru.wikipedia.org/wiki/1499_год
    https://ru.wikipedia.org/wiki/1420_год
    https://ru.wikipedia.org/wiki/14_октября
    https://ru.wikipedia.org/wiki/Коронация
    https://ru.wikipedia.org/wiki/Мадейра
    https://ru.wikipedia.org/wiki/1468_год
    https://ru.wikipedia.org/wiki/XV_век
    https://ru.wikipedia.org/wiki/Пир_(застолье)
    https://ru.wikipedia.org/wiki/1415_год
    https://ru.wikipedia.org/wiki/20_января
    https://ru.wikipedia.org/wiki/1349_год
    https://ru.wikipedia.org/wiki/1450_год
    https://ru.wikipedia.org/wiki/1481_год
    https://ru.wikipedia.org/wiki/1494_год
    https://ru.wikipedia.org/wiki/Александр
    https://ru.wikipedia.org/wiki/21_июля

https://ru.wikipedia.org/wiki/Чемпионат_Норвегии_по_кёрлингу_среди_смешанных_команд_2019
https://ru.wikipedia.org/wiki/3_мая
https://ru.wikipedia.org/wiki/Сергий_Радонежский
https://ru.wik